# Write elasticsearch index to single text file

Set variables

In [ ]:
es_host = "doccano-dataset-tools-es01"
es_index = "free-text-reports"
field = "text"
output_txt_file = '/home/jovyan/work/resources/data_ignored/data.txt'
output_model_file = "fasttext_rad.bin"
output_vec_file = "fasttext_rad.vec"

Install dependencies

In [ ]:
import sys
import re
sys.path.append("./../")

In [ ]:
pip install fasttext

Write .txt file of all documents in configured elasticsearch index

In [ ]:
from nlp_datau.index_to_txt import IndexToTxt
to_txt = IndexToTxt(es_host=es_host, es_index=es_index)
to_txt.write_index(output_txt_file, field)

Methods to clean text using SpaCy for sentence splitting (not sure if sentence splitting is required)

In [ ]:
import spacy

nlp = spacy.load("nl_core_news_sm")
from string import punctuation

def replace_punct(string):
    if string is None: 
        return ''
    for ch in ['&','#', '?', '!', ':', ';', ', ', '- ', '(' , ')', '[', ']']:
         if ch in string:
            string = string.replace(ch, ' ')
    return string

    
def clean(line):
    lines = []
    line = replace_punct(line)
    doc = nlp(line)
    for sent in doc.sents:
        line_sentence = sent.text
        line_sentence = line_sentence.lower()
        line_sentence = line_sentence.strip()
        line_sentence = line_sentence.rstrip('.')
        line_sentence = line_sentence.rstrip(',')

        if line_sentence is not None and len(line_sentence) > 0 and len(line_sentence.split()) > 2:
            lines.append(line_sentence)
    return lines

Clean text and write to file

In [ ]:
with open(output_txt_file + 'clean.txt', 'w') as out_file:
    with open(output_txt_file) as fp:
        for cnt, line in enumerate(fp):                
            cleaned_lines = clean(line)
            if cnt % 1000 == 0:
                print('cleaned_lines={}'.format(cnt))
            for cleaned_line in cleaned_lines:
                out_file.write(cleaned_line + '\n')
                
print('done') 